In [1]:
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
import time
from pyspark import SparkFiles

In [2]:
spark = SparkSession.builder.\
master("local").\
appName("Exercise 1").\
config("spark.some.config.option", "some-value").\
getOrCreate()

In [5]:
df1 = spark.read.option("inferSchema", "true").parquet("/home/saloni/Downloads/consumerInternet.parquet")

In [7]:
df1.show(2)

+-----+----------+------------+-----------------+--------------------+---------+--------------------+---------------+-------------+-------+
|Sr_No|      Date|Startup_Name|Industry_Vertical|         SubVertical|     City|      Investors_Name|InvestmentnType|Amount_in_USD|Remarks|
+-----+----------+------------+-----------------+--------------------+---------+--------------------+---------------+-------------+-------+
|  152|01/09/2018|     Netmeds|Consumer Internet|Online Pharmacy C...|  Chennai|Sistema Asia Fund...| Private Equity|  3,50,00,000|    nan|
|  154|03/09/2018|  Daily hunt|Consumer Internet|News and ebooks M...|Bengaluru|         Falcon Edge| Private Equity|    63,90,000|    nan|
+-----+----------+------------+-----------------+--------------------+---------+--------------------+---------------+-------------+-------+
only showing top 2 rows



In [9]:
df2 = spark.read.option("inferSchema", "true").option("header","true").csv("/home/saloni/Downloads/startup.csv")

In [10]:
df2.show(3)

+-----+----------+------------+-----------------+--------------------+---------+--------------------+--------------------+-------------+-------+
|Sr_No|      Date|Startup_Name|Industry_Vertical|         SubVertical|     City|      Investors_Name|     InvestmentnType|Amount_in_USD|Remarks|
+-----+----------+------------+-----------------+--------------------+---------+--------------------+--------------------+-------------+-------+
|    1|09/01/2020|      BYJU’S|           E-Tech|          E-learning|Bengaluru|Tiger Global Mana...|Private Equity Round| 20,00,00,000|   null|
|    2|13/01/2020|      Shuttl|   Transportation|App based shuttle...|  Gurgaon|Susquehanna Growt...|            Series C|    80,48,394|   null|
|    3|09/01/2020|   Mamaearth|       E-commerce|Retailer of baby ...|Bengaluru|Sequoia Capital I...|            Series B|  1,83,58,860|   null|
+-----+----------+------------+-----------------+--------------------+---------+--------------------+--------------------+--------

In [11]:
df1.registerTempTable("df1")
df2.registerTempTable("df2")
df = spark.sql("select * from df1 FULL OUTER JOIN df2 using(Sr_No,Date,Startup_Name,Industry_Vertical,SubVertical,City,Investors_Name,InvestmentnType,Amount_in_USD,Remarks) ;")
df = df.filter("Sr_No is not null")
df = df.dropDuplicates()
df.show(3)

+-----+----------+------------+-----------------+--------------------+---------+--------------------+---------------+-------------+-------+
|Sr_No|      Date|Startup_Name|Industry_Vertical|         SubVertical|     City|      Investors_Name|InvestmentnType|Amount_in_USD|Remarks|
+-----+----------+------------+-----------------+--------------------+---------+--------------------+---------------+-------------+-------+
|  751|14/06/2017|     MrNeeds|Consumer Internet|Micro Food Delive...|    Noida|undisclosed inves...|   Seed Funding|     5,00,000|    nan|
|  976|24/03/2017|   Delhivery|       Technology|Tech enabled Fulf...|New Delhi|Carlyle Group, Ti...| Private Equity| 10,00,00,000|    nan|
|  946|07/03/2017|Data Resolve|       Technology|Cyber Security So...|    Noida|IDFC Parampara Ea...| Private Equity|    10,00,000|    nan|
+-----+----------+------------+-----------------+--------------------+---------+--------------------+---------------+-------------+-------+
only showing top 3 r

In [12]:
#Q1 How many startups are there in Pune City?

df.registerTempTable("df")
q1 = spark.sql("select count(*) from df where city = 'Pune' ;")
q1.show()

+--------+
|count(1)|
+--------+
|     105|
+--------+



In [13]:
#Q2 How many startups in Pune got their Seed/ Angel Funding?

q2 = spark.sql("select count(*) from df where city = 'Pune' and ( InvestmentnType LIKE '%Seed%' or InvestmentnType LIKE '%Angel%') ;")
q2.show()

+--------+
|count(1)|
+--------+
|      58|
+--------+



In [14]:
#Q3 What is the total amount raised by startups in Pune City? Hint - use regex_replace to get rid of null

q3 = spark.sql("select sum(cast(NVL(Amount_in_USD,0) as double)) from df where city = 'Pune' ;")
q3.show()

+-----------------------------------------------+
|sum(CAST(nvl(df.`Amount_in_USD`, 0) AS DOUBLE))|
+-----------------------------------------------+
|                                           null|
+-----------------------------------------------+



In [15]:
#Q4 What are the top 5 Industry_Vertical which has the highest number of startups in India?

q4 = spark.sql("select Industry_Vertical, count(Startup_Name) cnt from df group by Industry_Vertical order by cnt desc limit 5 ; ")
q4.show()

+-----------------+---+
|Industry_Vertical|cnt|
+-----------------+---+
|Consumer Internet|941|
|       Technology|478|
|        eCommerce|186|
|              nan|171|
|       Healthcare| 70|
+-----------------+---+



In [22]:
#Q5 Find the top Investor(by amount) of each year.

q5 = spark.sql("select Investors_Name, year(date)as y, max(cast(Amount_in_USD as double)) from df group by Investors_Name,y;")
q5.show()

+--------------------+----+----------------------------------+
|      Investors_Name|   y|max(CAST(Amount_in_USD AS DOUBLE))|
+--------------------+----+----------------------------------+
|PBS Srinivas, Vin...|null|                              null|
|Alok Rawat, Grey ...|null|                              null|
|          Xiaomi Inc|null|                              null|
|      Blume ventures|null|                              null|
|      ResponsAbility|null|                              null|
|     RPG Enterprises|null|                              null|
|Kanwaljit Singh, ...|null|                              null|
|    The Phoenix Fund|null|                              null|
|Bharanidharan Vis...|null|                              null|
|Blume Ventures, M...|null|                              null|
|       SoftBank Corp|null|                              null|
|Matrix Partners I...|null|                              null|
|            Logtally|null|                            

In [18]:
#Q6 Which SubVertical had the highest growth(in number of startups) over the years?

q5 = spark.sql("select SubVertical, count(Startup_Name) cnt from df where Subvertical <> 'nan' group by SubVertical order by cnt desc limit 1;")
q5.show()

+--------------------+---+
|         SubVertical|cnt|
+--------------------+---+
|Online Lending Pl...| 11|
+--------------------+---+

